In [6]:
import os
import rasterio
from shapely.geometry import box
import geopandas as gpd
import pandas as pd
from fiona.crs import from_epsg
import pycrs
from matplotlib import pyplot

In [104]:
os.chdir('C://Users//Scott/Desktop/exampleData/')
os.getcwd()

'C:\\Users\\Scott\\Desktop\\exampleData'

In [105]:
fp = r"MD_510-N-PAYSON-ST_2014.tif"
#loadedGeotif = rasterio.open(fp)

In [106]:
epsg_code = int(loadedGeotif.crs.data['init'][5:])
print(epsg_code)

4326


In [107]:
def getNeighbors(geoTiff, bigdf): #geotiff, dataframe to parse, new dataframeName
    """Function to show image of vacant property"""
    
    loadedGeotif = rasterio.open(geoTiff)
    loadedGeotif.bounds
    #bounding box, assumes box is north south
    northlat = loadedGeotif.bounds[3]
    southlat = loadedGeotif.bounds[1]
    westLong = loadedGeotif.bounds[0]
    eastLong = loadedGeotif.bounds[2]
    
    newdf = bigdf[(bigdf['lat'] > southlat) & 
                            (bigdf['lat'] < northlat) &
                            (bigdf['lon'] < eastLong)&
                            (bigdf['lon'] > westLong)]
    return newdf
    #return df that has the addresses in the photo

In [108]:
# Set filepath (fix path relative to yours)
biggyShape = "C://Users//Scott/Desktop/OpenBaltimoreData/Real_Property/Real_Property.shp" #over a gig in size, takes time

# Read file using gpd.read_file()
realPropShape = gpd.read_file(biggyShape)
type(realPropShape)

geopandas.geodataframe.GeoDataFrame

In [93]:
realPropShape.head()

,OBJECTID,PIN,PINRELATE,BLOCKLOT,BLOCK,LOT,WARD,SECTION,ASSESSOR,TAXBASE,...,SUBTYPE_GE,SDATLINK,BLOCKPLAT,MAILTOADD,VACIND,ShapeSTAre,ShapeSTLen,geometry,lat,lon
0,1,0001001,0001001,0001 001,0001,001,15,370,329,141800,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"2045 W NORTH AVE, 21217",None,1359.932495,200.115609,"POLYGON ((1411050.735 598509.295, 1411050.905 ...",598466.720763,1.411045e+06
1,2,0001002,0001002,0001 002,0001,002,15,370,329,49500,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"2045 W NORTH AVE, 21217",None,1174.104431,195.673505,"POLYGON ((1411050.735 598509.295, 1411064.715 ...",598467.768920,1.411060e+06
2,3,0001003,0001003,0001 003,0001,003,15,370,308,7000,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"SUITE 201, 21208",Y,1188.327026,196.031880,"POLYGON ((1411083.735 598426.905, 1411069.505 ...",598468.355367,1.411074e+06
3,4,0001004,0001004,0001 004,0001,004,15,370,308,7000,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"9726 MENDOZA RD RANDALLSTOWN, MD, 21133",Y,1201.343567,196.338977,"POLYGON ((1411098.045 598427.565, 1411083.735 ...",598469.087418,1.411088e+06
4,5,0001005,0001005,0001 005,0001,005,15,370,308,21667,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"3112 MILFORD AVE, 21207",N,1106.556641,194.086967,"POLYGON ((1411111.235 598428.185, 1411098.045 ...",598469.721015,1.411102e+06


In [80]:
list(realPropShape)

['OBJECTID',
 'PIN',
 'PINRELATE',
 'BLOCKLOT',
 'BLOCK',
 'LOT',
 'WARD',
 'SECTION',
 'ASSESSOR',
 'TAXBASE',
 'BFCVLAND',
 'BFCVIMPR',
 'LANDEXMP',
 'IMPREXMP',
 'CITYCRED',
 'STATCRED',
 'CCREDAMT',
 'SCREDAMT',
 'PERMHOME',
 'ASSESGRP',
 'LOT_SIZE',
 'NO_IMPRV',
 'CURRLAND',
 'CURRIMPR',
 'EXMPLAND',
 'EXMPIMPR',
 'FULLCASH',
 'EXMPTYPE',
 'EXMPCODE',
 'USEGROUP',
 'ZONECODE',
 'SDATCODE',
 'ARTAXBAS',
 'DISTSWCH',
 'DIST_ID',
 'STATETAX',
 'CITY_TAX',
 'AR_OWNER',
 'DEEDBOOK',
 'DEEDPAGE',
 'SALEDATE',
 'OWNER_ABBR',
 'OWNER_1',
 'OWNER_2',
 'OWNER_3',
 'FULLADDR',
 'STDIRPRE',
 'ST_NAME',
 'ST_TYPE',
 'BLDG_NO',
 'FRACTION',
 'UNIT_NUM',
 'SPAN_NUM',
 'SPANFRAC',
 'ZIP_CODE',
 'EXTD_ZIP',
 'DHCDUSE1',
 'DHCDUSE2',
 'DHCDUSE3',
 'DHCDUSE4',
 'DWELUNIT',
 'EFF_UNIT',
 'ROOMUNIT',
 'RPDELTAG',
 'RESPAGCY',
 'SALEPRIC',
 'PROPDESC',
 'NEIGHBOR',
 'SRVCCNTR',
 'YEAR_BUILD',
 'STRUCTAREA',
 'LDATE',
 'OWNMDE',
 'GRNDRENT',
 'SUBTYPE_GE',
 'SDATLINK',
 'BLOCKPLAT',
 'MAILTOADD',
 'VACI

In [109]:
realPropShape = realPropShape.to_crs(epsg=4326) # convert projection to WGS84

In [110]:
#calc centroid of polygons
realPropLat = realPropShape.centroid.y
realPropLon = realPropShape.centroid.x
#add columns to gpd shapefile dataframe
realPropShape['lat'] = realPropLat
realPropShape['lon'] = realPropLon

In [111]:
realPropShape.head()

,OBJECTID,PIN,PINRELATE,BLOCKLOT,BLOCK,LOT,WARD,SECTION,ASSESSOR,TAXBASE,...,SUBTYPE_GE,SDATLINK,BLOCKPLAT,MAILTOADD,VACIND,ShapeSTAre,ShapeSTLen,geometry,lat,lon
0,1,0001001,0001001,0001 001,0001,001,15,370,329,141800,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"2045 W NORTH AVE, 21217",None,1359.932495,200.115609,"POLYGON ((-76.65112 39.30953, -76.65112 39.309...",39.309413,-76.651144
1,2,0001002,0001002,0001 002,0001,002,15,370,329,49500,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"2045 W NORTH AVE, 21217",None,1174.104431,195.673505,"POLYGON ((-76.65112 39.30953, -76.65107 39.309...",39.309416,-76.651090
2,3,0001003,0001003,0001 003,0001,003,15,370,308,7000,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"SUITE 201, 21208",Y,1188.327026,196.031880,"POLYGON ((-76.65101 39.30930, -76.65106 39.309...",39.309418,-76.651041
3,4,0001004,0001004,0001 004,0001,004,15,370,308,7000,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"9726 MENDOZA RD RANDALLSTOWN, MD, 21133",Y,1201.343567,196.338977,"POLYGON ((-76.65096 39.30931, -76.65101 39.309...",39.309420,-76.650990
4,5,0001005,0001005,0001 005,0001,005,15,370,308,21667,...,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"3112 MILFORD AVE, 21207",N,1106.556641,194.086967,"POLYGON ((-76.65091 39.30931, -76.65096 39.309...",39.309421,-76.650942


In [114]:
realPropShape1 = realPropShape[['FULLADDR','OWNER_1','ZIP_CODE','VACIND', 'YEAR_BUILD', 'SALEDATE', 'SALEPRIC', 'lat', 'lon']]
#realPropShape[1:16]

In [115]:
realPropShape1.head()

,FULLADDR,OWNER_1,ZIP_CODE,VACIND,YEAR_BUILD,SALEDATE,SALEPRIC,lat,lon
0,2045 W NORTH AVE,"CHONG, YON UN",21217,None,1920,11282005,5000,39.309413,-76.651144
1,2043 W NORTH AVE,"CHONG, YON UN",21217,None,0,11282005,5000,39.309416,-76.651090
2,2041 W NORTH AVE,NEW YORK INC,21217,Y,1915,10171975,0,39.309418,-76.651041
3,2039 W NORTH AVE,"MUHAMMAD, ANNA AVON",21217,Y,1920,11132014,0,39.309420,-76.650990
4,2037 W NORTH AVE,"PRICE, WALTER (LIFE)",21217,N,1920,06202018,21000,39.309421,-76.650942


In [116]:
closeby = getNeighbors(fp, realPropShape1)

In [117]:
closeby

,FULLADDR,OWNER_1,ZIP_CODE,VACIND,YEAR_BUILD,SALEDATE,SALEPRIC,lat,lon
6434,518 N PAYSON ST,"HENYA, KARNIEL",21223,None,1920,04032018,5000,39.294190,-76.649246
6435,516 N PAYSON ST,"RMX10, LLC",21223,None,1920,03022011,27000,39.294153,-76.649244
6436,514 N PAYSON ST,"KIDANE, TIKHER",21223,Y,1920,12222016,8000,39.294116,-76.649242
6437,512 N PAYSON ST,"KELLEY, ELIJAH",21223,Y,1920,02152018,1000,39.294079,-76.649240
6438,510 N PAYSON ST,"KELLEY, ELIJAH",21223,Y,1920,06072018,1000,39.294042,-76.649238
6439,508 N PAYSON ST,"KELLEY, ELIJAH (LIFE)",21223,None,1920,09112013,0,39.294004,-76.649236
6440,506 N PAYSON ST,"KELLEY, ELIJAH (LIFE)",21223,None,1920,11152012,0,39.293966,-76.649234
6441,504 N PAYSON ST,"KELLEY, ELIJAH (LIFE)",21223,None,1920,09112013,0,39.293928,-76.649232
6442,502 N PAYSON ST,JOYBAR CORPORATION,21223,Y,1920,07061992,14000,39.293891,-76.649230
6486,2000 W FRANKLIN ST,"WEBB, ARBIE D",21223,None,1920,10221991,0,39.293921,-76.649429
